#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2017


# Homework 3:  Classification Cook-off: Naive Bayes vs Rocchio (plus a little bit of recommenders)

### 100 points [5% of your final grade]

### Due: Wednesday, March 29 by 11:59pm

*Goals of this homework:* Hands-on practice building and evaluating classifiers.

*Submission Instructions:* To submit your homework, rename this notebook as YOUR_UIN_hw3.ipynb. Submit this notebook via eCampus. Your notebook should be completely self-contained, with the results visible in the notebook. 

*Late submission policy:* For this homework, you may use up to three of your late days, meaning that no submissions will be accepted after Saturday April 1 at 11:59pm.

# Part 0: Yelp review data

In this assignment, given a Yelp review, your task is to implement two classifiers to predict if the business category of this review is "food-relevant" or not, **only based on the review text**. The data is from the [Yelp Dataset Challenge](https://www.yelp.com/dataset_challenge).

## Build the training data

First, you will need to download this data file as your training data: [training_data.json](https://drive.google.com/open?id=0B_13wIEAmbQMdzBVTndwenoxQlk) 

The training data file includes 40,000 Yelp reviews. Each line is a json-encoded review, and **you should only focus on the "text" field**. As same as in homework 1, you should tokenize the review text by using the regular expression "\W+" (we discussed it in [this Piazza post](https://piazza.com/class/ixkk1fy863r1vs?cid=29). Do NOT remove stop words. **Do casefolding but no stemming**.

The label (class) information of each review is in the "label" field. It is **either "Food-relevant" or "Food-irrelevant"**.

## Testing data

We provide 100 yelp reviews here: [testing_data.json](https://drive.google.com/open?id=0B_13wIEAmbQMbXdyTkhrZDN4Wms). The testing data file has the same format as the training data file. Again, you can get the label informaiton in the "label" field. Only use it when you evalute your classifiers.

# Part 1: Naive Bayes classifier [35 points]

In this part, you will implement a Naive Bayes classifier, which outputs the probabilities that a given review belongs to each class.

Use a mixture model that mixes the probability from the document with the general collection frequency of the word. **You should use lambda = 0.7**. Be careful about the decimal rounding since multiplying many probabilities can generate a tiny value. We will not grade on the exact probability value, so feel free to change to logorithm summation (it's not required, though). If the tie case happens, **always go to the "Food-irrelevant" side**.

### What to report

* For the entire testing dataset, report the overall accuracy.
* For the class "Food-relevant", report the precision and recall.
* For the class "Food-irrelevant", report the precision and recall.

We will also grade on the quality of your code. So make sure that your code is clear and readable.

In [91]:
# Build the naive bayes classifier
# Insert as many cells as you want

import os
import re 
import json
import math
user_input = "/home/karthik/Downloads/training_data.json"
data=[]    #data is copied from json file into this list
f = open(user_input,'r')
for line in f:
    data.append(json.loads(line))
fi=0.0     #contains the number of food-irrelevant reviews in training data
fr=0.0     #contains the number of food-relevant reviews in training data
size_fi = 0.0 #contains the number of tokens in food-irrelevant class in training data including repeated tokens
size_fr = 0.0 #contains the number of tokens in food-relevant class in training data including repeated tokens
words_in_fr={} #dictionary to story unique tokens in food-relevant class in training data
words_in_fi={} #dictionary to story unique tokens in food-irrelevant class in training data
words_in_col={}#dictionary to story unique tokens in both the classes in training data
l = 0.7
i=0
col_len=0
for line in data:
    classs = line['label']
    if (classs == 'Food-irrelevant'):
        fi = fi + 1.0
        text = line['text']  #extracting the text part from each review and copying into text string
        text = text.lower()
        words_list = re.findall('\w+', text) #splitting it into each word
        for word in words_list:
            if word not in words_in_fi:
                words_in_fi[word] = 1.0  #adding words into food_irrelevant class dictionary
            else:
                words_in_fi[word] = words_in_fi[word] + 1.0  #finding the frequency of each word in food_irrelevant class
            if word not in words_in_col:
                words_in_col[word] = 1.0 #adding words into all class dictionary
            else:
                words_in_col[word] = words_in_col[word] + 1.0 #finding the frequency of each word in both classes
    else:
        fr = fr + 1.0
        text = line['text']
        text = text.lower()
        words_list = re.findall('\w+', text)
        for word in words_list:
            if word not in words_in_fr:
                words_in_fr[word] = 1.0 #adding words into food_relevant class dictionary
            else:
                words_in_fr[word] = words_in_fr[word] + 1.0 #finding the frequency of each word in food_relevant class
            if word not in words_in_col:
                words_in_col[word] = 1.0 #adding words into all class dictionary
            else:
                words_in_col[word] = words_in_col[word] + 1.0 #finding the frequency of each word in both classes
#print "size of words_in_fi is ",len(words_in_fi)," size of words_in_fr is ",len(words_in_fr)," size of words_in_col is ",len(words_in_col)
for word in words_in_fi:
    size_fi = size_fi + words_in_fi[word] #calculating size of fi class including repeated tokens
for word in words_in_fr:
    size_fr = size_fr + words_in_fr[word] #calculating size of fi class including repeated tokens
for word in words_in_col:
    col_len = col_len + words_in_col[word]

#for word in words_in_fi:  #calculating probability of each term using mixture model
#    words_in_fi[word] = l*((words_in_fi[word])/(size_fi)) + (1 - l)*((words_in_col[word])/(size_fi + size_fr))
#for word in words_in_fr:  #calculating probability of each term using mixture model
#    words_in_fr[word] = l*((words_in_fr[word])/(size_fr)) + (1 - l)*((words_in_col[word])/(size_fi + size_fr))
for word in words_in_col:  #calculating probability of each term using mixture model
    if word in words_in_fi:
        words_in_fi[word] = l*((words_in_fi[word])/(size_fi)) + (1 - l)*((words_in_col[word])/(size_fi + size_fr))
    else:
        words_in_fi[word] = (1 - l)*((words_in_col[word])/(size_fi + size_fr))
          
    if word in words_in_fr:  #calculating probability of each term using mixture model
        words_in_fr[word] = l*((words_in_fr[word])/(size_fr)) + (1 - l)*((words_in_col[word])/(size_fi + size_fr))    
    else:
        words_in_fr[word] = (1 - l)*((words_in_col[word])/(size_fi + size_fr))
#print "fi is ",size_fi," fr is ",size_fr," size is ",col_len," DISTINCT IS ",len(words_in_col)," len of fi is ",len(words_in_fi)," len of fr is ",len(words_in_fr)
        
#print "fi is ",fi," fr is ",fr
fi = fi + 0.0
#print "pr_fi is ",fi," pr_fr is ",fr
fi = fi/(fi + fr)  #probability of fi class
fr = 1.0 - fi #probability of fi class
#print "fi is ",fi," fr is ",fr
#print "pr_fi is ",fi," pr_fr is ",fr
#print "fi is ", size_fi," fr is ",size_fr," "

In [93]:
# Apply your classifier on the test data. Report the results.
# Insert as many cells as you want
test_input = "/home/karthik/Downloads/testing_data.json"
test_data=[]
f = open(test_input,'r')
for line in f:
    test_data.append(json.loads(line))
fo_i = 0
fo_r = 0
res_fi=0
res_fr=0
ii=0
rr=0
for line in test_data:
    
    test_text = line['text']   #extracting the text part from review
    test_text = test_text.lower() #converting to lower case
    test_words_list = re.split('\W+', test_text)
    test_words={}
    pr_fi=math.log(fi)
    pr_fr=math.log(fr)
    for word in test_words_list:
        if word not in test_words:
               test_words[word] = 1.0
        else:
            test_words[word] = test_words[word] + 1.0
    for word in test_words:
        if word in words_in_fi:
            pr_fi = pr_fi + math.log(words_in_fi[word]) * (test_words[word]) #recursively calculating probability for food-irrelevant class
            #print "pr_fi is ",pr_fi
        if word in words_in_fr:
            pr_fr = pr_fr + math.log(words_in_fr[word]) * (test_words[word]) #recursively calculating probability for food-relevant class
    if (pr_fi>= pr_fr):
        fo_i = fo_i + 1.0  #classifying as food-irrelevant
        if (line['label'] == 'Food-irrelevant'):
            ii = ii +1.0  #calculating true positives
    else:
        fo_r = fo_r + 1.0; #classifying as food-relevant
        if (line['label'] == 'Food-relevant'):
            rr = rr +1.0  #calculating true positives
    if (line['label'] == 'Food-irrelevant'):
        res_fi = res_fi + 1.0 #calculating actual no.of food-irrelevant
    else:
        res_fr = res_fr + 1.0 #calculating actual no.of food-relevant
#print "From my Classifier food irrelavant is ", fo_i ," ",ii," food relevant is ",fo_r," ",rr
#print "Correct numeber of food irrelavant is ", res_fi ," food relevant is ",res_fr
accuracy = (ii + rr)/100
precision_fi = ii/(fo_i)  #tp/(tp + fp)
recall_fi = ii/(res_fi)   #tp/(tp + fn)
precision_fr = rr/(fo_r)  #tp/(tp + fp)
recall_fr = rr/(res_fr)   #tp/(tp + fn)

print "Accuracy is ", accuracy*100,"%"
print "For Food-irrelevant class precision is ",precision_fi," recall is ",recall_fi
print "For Food-relevant class precision is ",precision_fr," recall is ",recall_fr


Accuracy is  89.0 %
For Food-irrelevant class precision is  0.862068965517  recall is  0.78125
For Food-relevant class precision is  0.901408450704  recall is  0.941176470588


# Part 2: Rocchio classifier [35 points]

In this part, your job is to implement a Rocchio classifier for "food-relevant vs. food-irrelevant". You need to aggregate all the reviews of each class, and find the center. **Use the normalized raw term frequency**.


### What to report

* For the entire testing dataset, report the overall accuracy.
* For the class "Food-relevant", report the precision and recall.
* For the class "Food-irrelevant", report the precision and recall.

We will also grade on the quality of your code. So make sure that your code is clear and readable.

In [95]:
# Build the Rocchio classifier
# Insert as many cells as you want

import os
import re 
import json
import math
user_input = "/home/karthik/Downloads/training_data.json"
data=[]    #data is copied from json file into this list
f = open(user_input,'r')
for line in f:
    data.append(json.loads(line))
fi=0     #contains the number of food-irrelevant reviews in training data
fr=0     #contains the number of food-relevant reviews in training data
cen_fi={} #centroid for food-irrelevant class
cen_fr={} #centroid for food-relevant class
l = 0.7
for line in data:
    classs = line['label']
    words_in_fr={} #dictionary to story unique tokens in food-relevant class in training data for each review
    words_in_fi={} #dictionary to story unique tokens in food-irrelevant class in training data for each review
    length = 0.0
    if (classs == 'Food-irrelevant'):
        fi = fi + 1.0
        text = line['text']  #extracting the text part from each review and copying into text string
        text = text.lower()
        words_list = re.findall(r"[\w]+", text) #splitting it into each word
        for word in words_list:
            if word not in words_in_fi:
                words_in_fi[word] = 1.0  #adding words into food_irrelevant class dictionary
            else:
                words_in_fi[word] = words_in_fi[word] + 1.0  #finding the frequency of each word in food_irrelevant class
        for word in words_in_fi:  #magnitude of the vector is calculated
            length = length + (words_in_fi[word] ** (2))  #numbers are squared
        length = length **(0.5)  #square root is taken
        for word in words_in_fi:
            words_in_fi[word] = words_in_fi[word]/length  #normalization is done
            if word in cen_fi:
                cen_fi[word] = cen_fi[word] + words_in_fi[word] #centroid is found by adding all the vectors average is done later
            else:
                cen_fi[word] = words_in_fi[word]
    else:  # the above same process is repeated for food-relavant class
        fr = fr + 1.0
        text = line['text']
        text = text.lower()
        words_list = re.findall(r"[\w]+", text)
        for word in words_list:
            if word not in words_in_fr:
                words_in_fr[word] = 1.0 #adding words into food_relevant class dictionary
            else:
                words_in_fr[word] = words_in_fr[word] + 1.0 #finding the frequency of each word in food_relevant class
        for word in words_in_fr:
            length = length + (words_in_fr[word] ** (2))
        length = length **(0.5)
        for word in words_in_fr:
            words_in_fr[word] = words_in_fr[word]/length
            if word in cen_fr:
                cen_fr[word] = cen_fr[word] + words_in_fr[word]
            else:
                cen_fr[word] = words_in_fr[word]
for word in cen_fi:
    cen_fi[word] = cen_fi[word]/fi
for word in cen_fr:
    cen_fr[word] = cen_fr[word]/fr
#print fi,fr

In [94]:
# Apply your classifier on the test data. Report the results.
# Insert as many cells as you want

test_input = "/home/karthik/Downloads/testing_data.json"
test_data=[]
f = open(test_input,'r')
for line in f:
    test_data.append(json.loads(line))
fo_i = 0
fo_r = 0
res_fi=0
res_fr=0
ii=0
rr=0
for line in test_data:
    length = 0.0
    distance_fi = 0.0 #distance from food-irrelevant class
    distance_fr = 0.0 #distance from food-relevant class
    if (line['label'] == 'Food-irrelevant'):
        res_fi = res_fi + 1
    else:
        res_fr = res_fr + 1
    test_text = line['text']  #extracting the text part from review
    test_text = test_text.lower() #converting to lower case
    test_words_list = re.findall(r"[\w]+", test_text)
    test_words={}
    for word in test_words_list:
        if word not in test_words:
               test_words[word] = 1.0
        else:
            test_words[word] = test_words[word] + 1.0
    for word in test_words:
        length = length + (test_words[word] ** 2)
    length = length **(0.5)  #finding the magnitude of test vector
    for word in test_words:
        test_words[word] = test_words[word]/length #normalizing test vector
        if word in cen_fi:
            distance_fi = distance_fi + ((test_words[word] - cen_fi[word])**(2)) #finding distance by subtracting if word is there in both centroid and test vector
        else:
            distance_fi = distance_fi + ((test_words[word] - 0)**(2)) #if word not in centroid just taking 0 in its place
        if word in cen_fr:
            distance_fr = distance_fr + ((test_words[word] - cen_fr[word])**(2)) #finding distance by subtracting if word is there in both centroid and test vector
        else:
            distance_fr = distance_fr + ((test_words[word] - 0)**(2)) #if word not in centroid just taking 0 in its place
    distance_fi = distance_fi ** (0.5)
    distance_fr = distance_fr ** (0.5)
    if (distance_fi >= distance_fr):
        fo_r = fo_r + 1 #classifying as food-irrelevant
        if (line['label'] == 'Food-relevant'):
            rr = rr +1.0 #true positive
    else:
        fo_i = fo_i + 1 #classifying as food-relevant
        if (line['label'] == 'Food-irrelevant'):
            ii = ii +1.0 #true positive
    
#print "food irrelavant is ", fo_i ," ",ii," food relevant is ",fo_r,rr
#print "food irrelavant is ", res_fi ," food relevant is ",res_fr

precision_fi = ii/(fo_i) #tp/(tp + fp)
recall_fi = ii/(res_fi)  #tp/(tp + fn)
precision_fr = rr/(fo_r) #tp/(tp + fp)
recall_fr = rr/(res_fr)  #tp/(tp + fn)

print "Accuracy is ",(ii + rr),"%"
print "For Food-irrelevant class precision is ",precision_fi," recall is ",recall_fi
print "For Food-relevant class precision is ",precision_fr," recall is ",recall_fr


Accuracy is  71.0 %
For Food-irrelevant class precision is  0.540540540541  recall is  0.625
For Food-relevant class precision is  0.809523809524  recall is  0.75


# Part 3: Naive Bayes vs. Rocchio [20 points]

Which method gives the better results? In terms of what? How did you compare them? Can you explain why you observe what you do? Write 1-3 paragraphs below.

**Add your answer here:**

Naive Bayes gave the best result. I used accuracy to compare these two. As we can say in Naive Bayes I was getting 89% accuracy where as in Rocchio I was getting 71% accuracy. Also the precision and recall is better for Naive Bayes compared to Rocchio. I think it is better because as the number of dimensions(words) increase then Naive Bayes gives a better result because of probabilities.

# Part 4: Recommenders [10 points]

Finally, since we've begun our discussion of recommenders, let's do a quick problem too:

The table below is a utility matrix, representing the ratings, on a 1–5 star scale, of eight items, *a* through *h*, by three users *A*, *B*, and *C*. 
<pre>

  | a  b  c  d  e  f  g  h
--|-----------------------
A | 4  5     5  1     3  2
B |    3  4  3  1  2  1
C | 2     1  3     4  5  3

</pre>

Compute the following from the data of this matrix.

(a) Treating the utility matrix as boolean, compute the Jaccard distance between each pair of users.

(b) Repeat Part (a), but use the cosine distance.

(c) Treat ratings of 3, 4, and 5 as 1 and 1, 2, and blank as 0. Compute the Jaccard distance between each pair of users.

(d) Repeat Part (c), but use the cosine distance.

(e) Normalize the matrix by subtracting from each nonblank entry the average
value for its user.

(f) Using the normalized matrix from Part (e), compute the cosine distance
between each pair of users.

(g) Which of the approaches above seems most reasonable to you? Give a one or two sentence argument supporting your choice.

**Add your answer here:**

(a) J(A,B) = J(B,C) = J(C,A) = 0.5

(b) Cos(A,B) = 0.6010 Cos(B,C) = 0.5139 Cos(C,A) = 0.6149  

(c) J(A,B) = J(B,C) = J(C,A) = 0.5

(d) Cos(A,B) = 0.5774 Cos(B,C) = 0.2887 Cos(C,A) = 0.5

(e)    a    b    c     d    e     f      g     h
   A 0.66  1.66       1.66 -2.33       -0.33  -1.33
   B       0.66  1.66 0.66 -1.33 -0.33 -1.33
   C -1          -2    0          1      2     0

(f) Cos(A,B) = 0.5835 Cos(B,C) = -0.7399 Cos(C,A) = -0.1147

(g) The normalized matrix approach seems most reasonable as a persons average can vary i.e he might give 3 as his least rating even when its not good and other person might give 1 as his least rating. So as this normalized approach takes care of it by subtracting the average of each user this is a better option